In [1]:
"""
LEME - Sprint 1, Dia 1: Criação do Dataset
Google Colab

Objetivo: Gerar 1000 perfis fictícios de usuários
Campos: área profissional, acessibilidade, tempo na plataforma, módulos concluídos
"""

import pandas as pd
import numpy as np
import json

np.random.seed(42)


In [2]:
# ============================================
# CONFIGURAÇÃO
# ============================================

# 6 Áreas profissionais
AREAS = ['Advogado', 'Design', 'Secretariado', 'Contabilidade', 'Logistica', 'SoftSkills']

# 3 Tipos de acessibilidade
ACESSIBILIDADES = ['cego', 'libras', 'nenhuma']

In [3]:
# ============================================
# CATÁLOGO DE TRILHAS (5 módulos por área)
# ============================================

TRILHAS = {
    'Advogado': {
        'nome': 'IA para Advogados',
        'modulos': [
            'Módulo 1: Introdução à IA Jurídica',
            'Módulo 2: Automação de Contratos',
            'Módulo 3: Análise de Jurisprudência com IA',
            'Módulo 4: Ética e Responsabilidade Digital',
            'Módulo 5: Prática com Ferramentas Jurídicas'
        ]
    },
    'Design': {
        'nome': 'IA para Designers',
        'modulos': [
            'Módulo 1: Fundamentos de IA Generativa',
            'Módulo 2: Ferramentas de Design com IA',
            'Módulo 3: Criação de Imagens e Protótipos',
            'Módulo 4: UX/UI Acessível',
            'Módulo 5: Portfolio Digital'
        ]
    },
    'Secretariado': {
        'nome': 'IA para Secretariado',
        'modulos': [
            'Módulo 1: Assistentes Virtuais e Automação',
            'Módulo 2: Gestão de Agenda e Tarefas',
            'Módulo 3: Comunicação Empresarial com IA',
            'Módulo 4: Organização de Documentos',
            'Módulo 5: Soft Skills para Secretariado'
        ]
    },
    'Contabilidade': {
        'nome': 'IA para Contabilidade',
        'modulos': [
            'Módulo 1: IA na Gestão Financeira',
            'Módulo 2: Análise de Dados Contábeis',
            'Módulo 3: Automação de Processos Fiscais',
            'Módulo 4: Auditoria com IA',
            'Módulo 5: Relatórios Inteligentes'
        ]
    },
    'Logistica': {
        'nome': 'IA para Logística',
        'modulos': [
            'Módulo 1: Otimização de Rotas com IA',
            'Módulo 2: Previsão de Demanda',
            'Módulo 3: Gestão de Estoque Inteligente',
            'Módulo 4: Rastreamento e IoT',
            'Módulo 5: Sustentabilidade Logística'
        ]
    },
    'SoftSkills': {
        'nome': 'Soft Skills Essenciais',
        'modulos': [
            'Módulo 1: Comunicação Eficaz',
            'Módulo 2: Liderança Inclusiva',
            'Módulo 3: Resolução de Problemas',
            'Módulo 4: Trabalho em Equipe',
            'Módulo 5: Adaptabilidade e Resiliência'
        ]
    }
}


# Criar versões acessíveis (mesmos módulos, mas com flag de acessibilidade)
TRILHAS_ACESSIVEIS = {}
for area, trilha in TRILHAS.items():
    TRILHAS_ACESSIVEIS[f'{area}_Acessivel'] = {
        'nome': f"{trilha['nome']} (Acessível)",
        'modulos': trilha['modulos']
    }


In [4]:
# ============================================
# FUNÇÃO: CALCULAR TAXA DE CONCLUSÃO
# ============================================

def calcular_taxa_conclusao(acessibilidade):
    """
    Calcula taxa de conclusão baseada na acessibilidade
    Usuários com acessibilidades tendem a ter taxas menores (barreiras reais)
    """
    taxa_base = 0.72

    # Penalidades por acessibilidade (simulando barreiras)
    if acessibilidade == 'cego':
        taxa_base -= 0.15
    elif acessibilidade == 'libras':
        taxa_base -= 0.12

    # Ruído aleatório (variação natural)
    taxa_final = taxa_base + np.random.normal(0, 0.08)
    return max(0.30, min(0.95, taxa_final))


In [5]:
# ============================================
# GERAR DATASET (1000 usuários)
# ============================================

n_usuarios = 1000
dados = []

# 30% com acessibilidades (representando inclusão)
n_com_acess = int(n_usuarios * 0.30)
n_sem_acess = n_usuarios - n_com_acess

print("="*60)
print("🔄 GERANDO DATASET...")
print("="*60)

# Gerar usuários COM acessibilidade
for i in range(n_com_acess):
    acessibilidade = np.random.choice(['cego', 'libras'])
    area = np.random.choice(AREAS)

    taxa = calcular_taxa_conclusao(acessibilidade)

    dados.append({
        'id': f'user_{i+1}',
        'area': area,
        'acessibilidade': acessibilidade,
        'taxa_conclusao': round(taxa, 3),
        'modulos_concluidos': np.random.randint(0, 12),
        'tempo_plataforma_dias': np.random.randint(1, 120)
    })

# Gerar usuários SEM acessibilidade
for i in range(n_sem_acess):
    area = np.random.choice(AREAS)

    taxa = calcular_taxa_conclusao('nenhuma')

    dados.append({
        'id': f'user_{n_com_acess + i + 1}',
        'area': area,
        'acessibilidade': 'nenhuma',
        'taxa_conclusao': round(taxa, 3),
        'modulos_concluidos': np.random.randint(0, 20),
        'tempo_plataforma_dias': np.random.randint(1, 180)
    })

# Criar DataFrame
df = pd.DataFrame(dados)

# Adicionar label: trilha ideal (com ou sem acessibilidade)
def atribuir_trilha_ideal(row):
    """Se tem acessibilidade, sugere versão acessível"""
    if row['acessibilidade'] != 'nenhuma':
        return f"{row['area']}_Acessivel"
    return row['area']

df['trilha_ideal'] = df.apply(atribuir_trilha_ideal, axis=1)

# Embaralhar dados
df = df.sample(frac=1).reset_index(drop=True)

🔄 GERANDO DATASET...


In [6]:
# ============================================
# SALVAR ARQUIVOS
# ============================================

# Salvar CSV
df.to_csv('leme_dataset.csv', index=False)

# Salvar catálogo de trilhas (JSON)
catalogo_completo = {**TRILHAS, **TRILHAS_ACESSIVEIS}
with open('leme_trilhas.json', 'w', encoding='utf-8') as f:
    json.dump(catalogo_completo, f, indent=2, ensure_ascii=False)


In [7]:
# ============================================
# RESUMO
# ============================================

print("✅ DATASET CRIADO COM SUCESSO!")
print("="*60)
print(f"\n📊 Estatísticas:")
print(f"  Total de usuários: {len(df)}")
print(f"  Com acessibilidade: {len(df[df['acessibilidade'] != 'nenhuma'])} ({len(df[df['acessibilidade'] != 'nenhuma'])/len(df)*100:.1f}%)")
print(f"  Sem acessibilidade: {len(df[df['acessibilidade'] == 'nenhuma'])} ({len(df[df['acessibilidade'] == 'nenhuma'])/len(df)*100:.1f}%)")

print(f"\n📈 Taxa média de conclusão:")
print(f"  Geral: {df['taxa_conclusao'].mean():.2%}")
print(f"  COM acessibilidade: {df[df['acessibilidade'] != 'nenhuma']['taxa_conclusao'].mean():.2%}")
print(f"  SEM acessibilidade: {df[df['acessibilidade'] == 'nenhuma']['taxa_conclusao'].mean():.2%}")

print(f"\n🎯 Distribuição por área:")
print(df['area'].value_counts().to_string())

print(f"\n♿ Distribuição de acessibilidade:")
print(df['acessibilidade'].value_counts().to_string())

print(f"\n✅ Arquivos salvos:")
print("  - leme_dataset.csv (1000 perfis)")
print("  - leme_trilhas.json (12 trilhas: 6 padrão + 6 acessíveis)")

print(f"\n📋 Primeiros 5 registros:")
print(df.head())

✅ DATASET CRIADO COM SUCESSO!

📊 Estatísticas:
  Total de usuários: 1000
  Com acessibilidade: 300 (30.0%)
  Sem acessibilidade: 700 (70.0%)

📈 Taxa média de conclusão:
  Geral: 68.31%
  COM acessibilidade: 58.58%
  SEM acessibilidade: 72.48%

🎯 Distribuição por área:
area
Advogado         189
Logistica        175
Secretariado     175
SoftSkills       169
Contabilidade    150
Design           142

♿ Distribuição de acessibilidade:
acessibilidade
nenhuma    700
cego       156
libras     144

✅ Arquivos salvos:
  - leme_dataset.csv (1000 perfis)
  - leme_trilhas.json (12 trilhas: 6 padrão + 6 acessíveis)

📋 Primeiros 5 registros:
         id           area acessibilidade  taxa_conclusao  modulos_concluidos  \
0  user_301       Advogado        nenhuma           0.920                  16   
1  user_167      Logistica         libras           0.615                   3   
2  user_610  Contabilidade        nenhuma           0.742                   1   
3  user_436       Advogado        nenhum